In [2]:
import io
import keras
import tensorflow as tf
import os
import numpy as np # used for handling numbers
import pandas as pd # used for handling the dataset
df= pd.read_csv('drive/My Drive/Colab Notebooks/dataset/labeled_outs.txt',delimiter=",",header=None)
# Dataset is now stored in a Pandas Dataframe

Using TensorFlow backend.


In [0]:
file = open('drive/My Drive/Colab Notebooks/dataset/vocab.txt', 'r')
# read all text
vocab = file.readlines()
vocabulary={}
for w in vocab:
  vocabulary[w.strip()]="a"
# close the file
file.close()

In [0]:
df.columns=['label','text']
df.text=df.text.astype(str)
df.label=df.label.astype(str)
df=df.sample(frac=1)
docs=df['text'].values
labels=df['label'].values

In [0]:
docs_filtered=[]
for d in docs:
  # split into tokens by white space
  tokens = d.split()
  temp=""
  for t in tokens:
    if t in vocabulary:
      temp=temp+" "+t
  docs_filtered.append(temp)
  


In [0]:
# integer encode the documents
from keras.preprocessing.text import one_hot
vocab_size = len(vocabulary)+1

encoded_docs = [one_hot(d, vocab_size) for d in docs_filtered]

In [7]:
# pad documents to a max length of 4 words
max_length = 200
padded_docs = keras.preprocessing.sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))

91795


In [8]:
#import sys
#import numpy
#numpy.set_printoptions(threshold=sys.maxsize)
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(dummy_y.shape)

(91795, 8)


In [9]:
emb_dim=32
emb_model = tf.keras.Sequential([ 
    tf.keras.layers.Input(shape=(max_length,)) ,                           
    tf.keras.layers.Embedding(vocab_size, output_dim=emb_dim),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(64,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(32,return_sequences=True,dropout=0.3),
    tf.keras.layers.GRU(16),
    tf.keras.layers.Dense(8, activation="softmax")
])

emb_model.compile(optimizer='adam',loss= 'categorical_crossentropy',metrics=['acc'])
print(emb_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           8000032   
_________________________________________________________________
gru (GRU)                    (None, 200, 128)          62208     
_________________________________________________________________
gru_1 (GRU)                  (None, 200, 64)           37248     
_________________________________________________________________
gru_2 (GRU)                  (None, 200, 32)           9408      
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                2400      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
Total params: 8,111,432
Trainable params: 8,111,432
Non-trainable params: 0
______________________________________________

In [10]:
emb_model.fit(padded_docs, dummy_y,epochs=30,batch_size=256, verbose=1,validation_split=0.1)

Epoch 1/30
323/323 [==============================] - 31s 95ms/step - loss: 1.7817 - acc: 0.3056 - val_loss: 1.6652 - val_acc: 0.3524
Epoch 2/30
323/323 [==============================] - 29s 91ms/step - loss: 1.1340 - acc: 0.5890 - val_loss: 0.8225 - val_acc: 0.7331
Epoch 3/30
323/323 [==============================] - 29s 91ms/step - loss: 0.5318 - acc: 0.8371 - val_loss: 0.3941 - val_acc: 0.8903
Epoch 4/30
323/323 [==============================] - 29s 90ms/step - loss: 0.2427 - acc: 0.9364 - val_loss: 0.3205 - val_acc: 0.9087
Epoch 5/30
323/323 [==============================] - 29s 91ms/step - loss: 0.1459 - acc: 0.9636 - val_loss: 0.2978 - val_acc: 0.9176
Epoch 6/30
323/323 [==============================] - 29s 90ms/step - loss: 0.0978 - acc: 0.9764 - val_loss: 0.3324 - val_acc: 0.9117
Epoch 7/30
323/323 [==============================] - 29s 90ms/step - loss: 0.0693 - acc: 0.9840 - val_loss: 0.3226 - val_acc: 0.9192
Epoch 8/30
323/323 [==============================] - 29s 90ms

In [11]:
emb_model1 = tf.keras.Model(inputs=emb_model.inputs , outputs=emb_model.layers[3].output)
emb_X = emb_model1.predict(padded_docs)
print(emb_X.shape)
#emb_X=np.reshape(emb_X, (emb_X.shape[0],1,-1))

(91795, 200, 32)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emb_X, dummy_y, test_size=0.18)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(75271, 200, 32)
(75271, 8)
(16524, 200, 32)
(16524, 8)


In [0]:

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv1D, MaxPooling1D,GlobalAveragePooling1D

In [0]:

model = Sequential()
model.add(Conv1D(filters=256, kernel_size=7,input_shape=(X_train.shape[1],X_train.shape[2]), activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.4))
model.add(Conv1D(filters=128, kernel_size=6, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.1))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [24]:

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['acc'])
# summarize the model
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 194, 256)          57600     
_________________________________________________________________
dropout_8 (Dropout)          (None, 194, 256)          0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 194, 256)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 194, 256)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 189, 128)          196736    
_________________________________________________________________
dropout_10 (Dropout)         (None, 189, 128)          0         
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 189, 128)         

In [25]:
history = model.fit(X_train, y_train, epochs=25,shuffle=False, verbose=1,validation_split=0.1)
#model.save_weights('./checkpoints/text_classifier_rnn_tfidf')
!mkdir -p saved_model
model.save('saved_model/text_classifier_rnn_tfidf')

Train on 67743 samples, validate on 7528 samples
Epoch 1/25
67743/67743 [==============================] - 17s 245us/step - loss: 0.0959 - acc: 0.9774 - val_loss: 0.0785 - val_acc: 0.9825
Epoch 2/25
67743/67743 [==============================] - 16s 241us/step - loss: 0.0740 - acc: 0.9831 - val_loss: 0.0864 - val_acc: 0.9826
Epoch 3/25
67743/67743 [==============================] - 16s 241us/step - loss: 0.0630 - acc: 0.9859 - val_loss: 0.0812 - val_acc: 0.9853
Epoch 4/25
67743/67743 [==============================] - 16s 240us/step - loss: 0.0520 - acc: 0.9889 - val_loss: 0.0692 - val_acc: 0.9875
Epoch 5/25
67743/67743 [==============================] - 16s 240us/step - loss: 0.0497 - acc: 0.9889 - val_loss: 0.0676 - val_acc: 0.9878
Epoch 6/25
67743/67743 [==============================] - 16s 240us/step - loss: 0.0499 - acc: 0.9892 - val_loss: 0.0721 - val_acc: 0.9865
Epoch 7/25
67743/67743 [==============================] - 16s 240us/step - loss: 0.0481 - acc: 0.9895 - val_loss: 0.0

In [26]:
results = model.evaluate(X_test, y_test)
print(results)

16524/16524 [==============================] - 2s 99us/step
[0.06512387010355214, 0.989348828792572]


In [0]:
y_pred=model.predict(X_test)

In [28]:
"""
import numpy as np
array1=[[ 0,  1,  2,  3,  4],
       [ 0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0],
       [5, 6, 7, 1, 0]]
array2=[[ 5, 6, 7, 1, 0],
       [ 0,  1,  2,  3,  4],
       [0,  1,  2,  3,  4],
       [5, 6, 7, 1, 0]]
"""
no_cat=len(y_test[0])
conf_mat=np.zeros(no_cat*no_cat).reshape(no_cat,no_cat)
test_size=len(y_test)
print(conf_mat.shape)
for i in range(test_size):
  y_t=y_test[i]
  y_p=y_pred[i]
  class_t=np.where(y_t == np.amax(y_t))
  class_t=class_t[0][0]
  class_p=np.where(y_p == np.amax(y_p))
  class_p=class_p[0][0]
  conf_mat[class_t][class_p]=conf_mat[class_t][class_p]+1

(8, 8)


In [29]:
tp=np.zeros(no_cat)
fp=np.zeros(no_cat)
tn=np.zeros(no_cat)
fn=np.zeros(no_cat)

prec=np.zeros(no_cat)
rec=np.zeros(no_cat)
f_score=np.zeros(no_cat)

for i in range(no_cat):
  for j in range(no_cat):
    if i==j:
      tp[i]=conf_mat[i][j]
    if i!= j:
      fp[i] = fp[i]+conf_mat[j][i]
      fn[i] = fn[i]+conf_mat[i][j]
    for k in range(no_cat):
      if i!=k & j!=k:
        tn[i]=tn[i]+conf_mat[j][k]

for i in range(no_cat):
    prec[i] = tp[i] * 100.0 / (tp[i] + fp[i])
    rec[i] = tp[i] * 100.0 / (tp[i] + fn[i])
    f_score[i] = 2 * prec[i] * rec[i] / (prec[i] + rec[i])
print(np.average(prec))
print(np.average(rec))
print(np.average(f_score))

98.88495402518008
98.93115207633198
98.9047963314457
